In [1]:
import matplotlib.pyplot as plt 
import numpy as np 
import os,time
import glob,sys

from astropy.io import fits
from astropy.visualization import ZScaleInterval
from mpl_toolkits.axes_grid1 import make_axes_locatable

from RetrieveSource import *
from EstimateBackground import *
from TractorTools import *
from Heasarc import *
from PSF import *


from matplotlib import rcParams
rcParams['font.family'] = 'serif'
rcParams['font.sans-serif'] = ['Times New Roman']

### Get image and catalog

In [2]:
# Check defaults of this function to point to correct file paths. 
uvfilter = 'um2'
id = 40417
segment = 1
extension = 1
path = "/home/bethany/Desktop/Swift_Astrometry"
galaxy = 'SMC'
savepath = path + f'/Photometry/{galaxy}/'

savepath = savepath + str(id)

if not os.path.exists(savepath):
    os.makedirs(savepath)

savefile = savepath + f"{id}_{segment}_{extension}_{uvfilter}.fits"

if uvfilter == 'um2':
    heafilter = 'UVM2'
    
def get_file(id,galaxy,segment=segment,extension=extension,filter=uvfilter,file_location = path):
    labels = ['RAhr','DEdeg','Umag','e_Umag','Bmag','e_Bmag','Vmag','e_Vmag','Imag','e_Imag','Flag','Jmag','e_Jmag','Hmag','e_Hmag','Ksmag','e_Ksmag']
    fname = glob.glob(f"{file_location}/{galaxy.upper()}/sw000{id}00{segment}{filter}_sk_{id}_{segment}_{extension}.new")
    cname = glob.glob(f"{file_location}/{galaxy.upper()}/sw000{id}00{segment}{filter}_sk_{id}_{segment}_{extension}.full.dat")
    HDR = fits.open(fname[0])
    CAT = pd.read_csv(cname[0],delimiter='\s+',names=labels)
    return HDR[0],CAT,cname

hdr,cat,catname = get_file(id=id,galaxy=galaxy)

### Run Tractor

In [ ]:
start = time.time()
print(f"File {id}_{segment}_{extension}_{uvfilter} now running.")

    
print('---Getting Catalog---')
source = get_meta().with_hdu(hdu=hdr,
                 usno_catalog='smc_anti_match_USNO_inf.dat',
                 optical_catalog=catname[0],
                 Umag_cutoff=np.nan,
                 Bmag_cutoff=np.nan,
                 fits_origin=0,
                 aperture_size=2.5*2,
                 xdim=[0,np.shape(hdr.data)[1]],  #  <---- Full image is selected here.
                 ydim=[0,np.shape(hdr.data)[0]])


print('---Getting Background---')
bkgd = BkgdEstimator(source,n_pix = [20,20])


print('---Getting PSF---')
pix_scale = np.abs(source.cdelt)*3600.

psf_object = psf_fit(pixel_per_arsecond = 1/pix_scale,
                     uvfilter = uvfilter, width = 23).psf


print('---Running Tractor---')

fwhm = 2.5
sigma =  fwhm/2.355
# Restrict by a factor of 4.
sigma = sigma/4.
        
TractorObject = PhotometryTools(source,
                    psf_filename = psf_object,
                    fits_table_sname = savefile, 
                    background = bkgd,
                    fit_positions = 0.05,
                    threshold = 1.5,
                    save_output=True)


print(f"Time Taken for Tractor: {(time.time()-start)/60/60} hours")

File 40417_1_1_um2 now running.
---Getting Catalog---


the RADECSYS keyword is deprecated, use RADESYSa. [astropy.wcs.wcs]
Set MJD-OBS to 55467.458785 from DATE-OBS.
Set MJD-END to 55467.461956 from DATE-END'. [astropy.wcs.wcs]
the RADECSYS keyword is deprecated, use RADESYSa. [astropy.wcs.wcs]
Set MJD-OBS to 55467.458785 from DATE-OBS.
Set MJD-END to 55467.461956 from DATE-END'. [astropy.wcs.wcs]
the RADECSYS keyword is deprecated, use RADESYSa. [astropy.wcs.wcs]
Set MJD-OBS to 55467.458785 from DATE-OBS.
Set MJD-END to 55467.461956 from DATE-END'. [astropy.wcs.wcs]


INFO: 
                Inconsistent SIP distortion information is present in the FITS header and the WCS object:
                SIP coefficients were detected, but CTYPE is missing a "-SIP" suffix.
                astropy.wcs is using the SIP distortion coefficients,
                therefore the coordinates calculated here might be incorrect.

                If you do not want to apply the SIP distortion coefficients,
                please remove the SIP coefficients from the FITS header or the
                WCS object.  As an example, if the image is already distortion-corrected
                (e.g., drizzled) then distortion components should not apply and the SIP
                coefficients should be removed.

                While the SIP distortion coefficients are being applied here, if that was indeed the intent,
                for consistency please append "-SIP" to the CTYPE in the FITS header or the WCS object.

                 [astropy.wcs.wcs]


/home/bethany/Desktop/SUMS/Tractor/RetrieveSource.py:353: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  drop_cat['phot'] = phot


---Getting Background---
Masked Sigma Clipped Stats Original Data: 0.005640092305839062 0.005121281370520592 0.003445275826379657


Masked Sigma Clipped Stats Back Sub Data: 0.000980092909730014 0.000552631775661172 0.0032253361204622804
---Getting PSF---
22.903962009432647
23
Scaling is 1.0041930732564 ArcSeconds per Pixel
Image needs to be 23 pixels to be 23 ArcSeconds in size.
23
Sum Before Normalization  616.9218729335043
Sum After Normalization  0.9999999999999998
---Running Tractor---
dlnp 35431138.07418785


In [ ]:
# Visualize Tractor results

f, axs = plt.subplots(2,3,figsize=(30,20))


vmin,vmax = ZScaleInterval().get_limits(source.data)
im1 = axs[0,0].imshow(source.data,vmin=vmin,vmax=vmax,origin='lower',cmap='Greys_r')

vmin,vmax = ZScaleInterval().get_limits(bkgd.background)
im2 = axs[0,1].imshow(bkgd.background ,vmin=vmin,vmax=vmax,origin='lower',cmap='Greys_r')

vmin,vmax = ZScaleInterval().get_limits(TractorObject.image)
im3 = axs[0,2].imshow(TractorObject.image,vmin=vmin,vmax=vmax,origin='lower',cmap='Greys_r')


axs[0,0].set_title('Original Data',fontsize=30)
axs[0,1].set_title('Background',fontsize=30)
axs[0,2].set_title('Tractor Data',fontsize=30)


vmin,vmax = ZScaleInterval().get_limits(TractorObject.model)
im4 = axs[1,0].imshow(TractorObject.model,vmin=vmin,vmax=vmax,origin='lower',cmap='Greys_r')

vmin,vmax = ZScaleInterval().get_limits(TractorObject.chi)
im5 = axs[1,1].imshow(TractorObject.chi ,vmin=vmin,vmax=vmax,origin='lower',cmap='Greys_r')

vmin,vmax = ZScaleInterval().get_limits(TractorObject.image-TractorObject.model)
im6 = axs[1,2].imshow(TractorObject.image-TractorObject.model ,vmin=vmin,vmax=vmax,origin='lower',cmap='Greys_r')

plt.colorbar(im1,ax=axs[0,0],orientation='vertical',fraction=0.046, pad=0.04)
plt.colorbar(im2,ax=axs[0,1],orientation='vertical',fraction=0.046, pad=0.04)
plt.colorbar(im3,ax=axs[0,2],orientation='vertical',fraction=0.046, pad=0.04)
plt.colorbar(im4,ax=axs[1,0],orientation='vertical',fraction=0.046, pad=0.04)
plt.colorbar(im5,ax=axs[1,1],orientation='vertical',fraction=0.046, pad=0.04)
plt.colorbar(im6,ax=axs[1,2],orientation='vertical',fraction=0.046, pad=0.04)

axs[1,0].set_title('Model',fontsize=30)
axs[1,1].set_title('Chi',fontsize=30)
axs[1,2].set_title('Residual',fontsize=30)

plt.savefig(savefile[:-5]+'_ScaledToSelf.png')

In [ ]:
# Visualize Tractor results

f, axs = plt.subplots(2,3,figsize=(30,20))

vmin,vmax = ZScaleInterval().get_limits(TractorObject.image)

im1 = axs[0,0].imshow(source.data,vmin=vmin,vmax=vmax,origin='lower',cmap='Greys_r')
im2 = axs[0,1].imshow(bkgd.background ,vmin=vmin,vmax=vmax,origin='lower',cmap='Greys_r')
im3 = axs[0,2].imshow(TractorObject.image,vmin=vmin,vmax=vmax,origin='lower',cmap='Greys_r')
im4 = axs[1,0].imshow(TractorObject.model,vmin=vmin,vmax=vmax,origin='lower',cmap='Greys_r')
im5 = axs[1,1].imshow(TractorObject.chi ,vmin=vmin,vmax=vmax,origin='lower',cmap='Greys_r')
im6 = axs[1,2].imshow(TractorObject.image-TractorObject.model ,vmin=vmin,vmax=vmax,origin='lower',cmap='Greys_r')

plt.colorbar(im1,ax=axs[0,0],orientation='vertical',fraction=0.046, pad=0.04)
plt.colorbar(im2,ax=axs[0,1],orientation='vertical',fraction=0.046, pad=0.04)
plt.colorbar(im3,ax=axs[0,2],orientation='vertical',fraction=0.046, pad=0.04)
plt.colorbar(im4,ax=axs[1,0],orientation='vertical',fraction=0.046, pad=0.04)
plt.colorbar(im5,ax=axs[1,1],orientation='vertical',fraction=0.046, pad=0.04)
plt.colorbar(im6,ax=axs[1,2],orientation='vertical',fraction=0.046, pad=0.04)

axs[0,0].set_title('Original Data',fontsize=30)
axs[0,1].set_title('Background',fontsize=30)
axs[0,2].set_title('Tractor Data',fontsize=30)
axs[1,0].set_title('Model',fontsize=30)
axs[1,1].set_title('Chi',fontsize=30)
axs[1,2].set_title('Residual',fontsize=30)

plt.savefig(savefile[:-5]+'_ScaledToTractorData.png')

In [ ]:
# Step 5: Run Heasarc 
Magnitudes = HeasarcRoutines(savefile,heafilter)

